# Lab 3: Google Earth Engine & Sentinel-2 Acquisition (2 Hours)

## ⏱️ Time Allocation (3 × 40 min modules with breaks)

**Module 1 (40 min):** GEE Setup & Authentication
- 10 min: Introduction to Google Earth Engine
- 25 min: GEE authentication and setup
- 5 min: Test GEE connection

**Break (10 min)**

**Module 2 (40 min):** Data Discovery
- 15 min: Define AOI in Iceland
- 20 min: Query Sentinel-2 image collection
- 5 min: Filter by cloud cover

**Break (10 min)**

**Module 3 (40 min):** Visualization & Export
- 15 min: Visualize RGB and false color
- 20 min: Export and download workflow
- 5 min: Advanced filtering (optional)

## 🎯 Learning Objectives

### Core (Essential - Everyone Should Complete)
- ✅ Authenticate with Google Earth Engine
- ✅ Define AOI in Iceland (Þingvellir region)
- ✅ Query Sentinel-2 image collection
- ✅ Filter by cloud cover (<20%)
- ✅ Visualize RGB and false color composites
- ✅ Export and download 4 scenes

### Optional (For Early Finishers)
- 🔵 Calculate spectral indices (NDVI, NDWI, NBR)
- 🔵 Advanced cloud masking with QA bands
- 🔵 Temporal analysis and time series
- 🔵 Multiple AOI comparison
- 🔵 Custom band combinations

### ⚠️ Pre-Lab Requirement
**You MUST have a Google Earth Engine account approved before this lab!**
- Apply at: https://earthengine.google.com/signup
- Approval takes 1-2 days
- Use your university or personal Gmail account

---

## Section 1: Introduction to Google Earth Engine (10 min)

### What is Google Earth Engine (GEE)?
Google Earth Engine is a cloud-based platform for planetary-scale geospatial analysis:
- **Petabyte-scale** catalog of satellite imagery
- **Server-side processing** - no downloads needed for analysis
- **Free** for research and education
- **Python and JavaScript APIs**

### Why GEE for This Course?
- ✅ Easy access to Sentinel-2 archive
- ✅ Built-in cloud filtering
- ✅ Fast querying and visualization
- ✅ No storage needed until export

### GEE Data Catalog
- Sentinel-2 (10m optical)
- Landsat 5/7/8/9 (30m optical)
- MODIS (250m-1km)
- Sentinel-1 (10m SAR)
- And many more!

### Sentinel-2 in GEE
- **Collection:** `COPERNICUS/S2_SR` (Surface Reflectance)
- **Bands:** 13 spectral bands (10m, 20m, 60m resolution)
- **Coverage:** Global, since 2015
- **Revisit:** 5 days (with both satellites)

## Section 2: GEE Setup and Authentication (5 min)

### Step 1: Install Earth Engine API

In [ ]:
# Install required packages (run once)
!pip install earthengine-api geemap folium geopandas

### Step 2: Authenticate GEE
You'll need to authenticate once per environment:

In [ ]:
import ee

# Trigger authentication (will open browser)
try:
    ee.Initialize()
    print("✅ Already authenticated!")
except:
    print("🔐 Authentication required...")
    ee.Authenticate()
    ee.Initialize()
    print("✅ Authentication successful!")

**Authentication Steps:**
1. Click the link that appears
2. Sign in with your Google account
3. Grant Earth Engine permissions
4. Copy the authorization code
5. Paste it back into the notebook

In [ ]:
# Import additional libraries
import geemap
import folium
import datetime
import pandas as pd
from pathlib import Path

print("✅ All libraries loaded successfully!")

## Section 3: Define Area of Interest (AOI) in Iceland (8 min)

### Choose Your Study Area
For this course, we'll focus on areas in Iceland with diverse land cover:
- **Reykjavik Region:** Urban + coastal
- **Þingvellir:** Vegetation + volcanic
- **Vatnajökull:** Glaciers + bare rock

Let's define a study area near **Þingvellir National Park**:

In [ ]:
# Define AOI as a bounding box (lon/lat)
# Þingvellir area: [west, south, east, north]
aoi_coords = [-21.3, 64.2, -21.0, 64.4]

# Create GEE geometry
aoi = ee.Geometry.Rectangle(aoi_coords)

print(f"AOI Center: {aoi.centroid().coordinates().getInfo()}")
print(f"AOI Area: {aoi.area().divide(1e6).getInfo():.2f} km²")

In [ ]:
# Visualize AOI on interactive map
Map = geemap.Map(center=[64.3, -21.15], zoom=10)
Map.addLayer(aoi, {'color': 'red'}, 'AOI')
Map.add_basemap('SATELLITE')
Map

### Alternative: Draw Your Own AOI
You can also draw directly on the map:

In [ ]:
# Interactive AOI drawing
Map = geemap.Map(center=[64.3, -21.15], zoom=10)
Map.add_basemap('SATELLITE')

# Instructions:
print("📍 Use the drawing tools to define your AOI:")
print("   1. Click the rectangle/polygon tool on the left")
print("   2. Draw your area of interest")
print("   3. Run the next cell to extract coordinates")

Map

In [ ]:
# Extract drawn AOI (if you used drawing tools)
# aoi = Map.draw_last_feature.geometry()
# print(f"Custom AOI coordinates: {aoi.bounds().getInfo()['coordinates']}")

## Section 4: Query Sentinel-2 Image Collection (12 min)

### Define Search Parameters

In [ ]:
# Date range: Summer 2024 (less snow, more vegetation)
start_date = '2024-06-01'
end_date = '2024-09-30'

# Cloud cover threshold
max_cloud_cover = 20  # percent

print(f"🔍 Searching for Sentinel-2 scenes:")
print(f"   Date Range: {start_date} to {end_date}")
print(f"   Max Cloud Cover: {max_cloud_cover}%")
print(f"   AOI: {aoi.area().divide(1e6).getInfo():.2f} km²")

### Query Image Collection

In [ ]:
# Query Sentinel-2 Surface Reflectance (Level 2A)
collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(aoi)
              .filterDate(start_date, end_date)
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover)))

# Get collection size
count = collection.size().getInfo()
print(f"\n✅ Found {count} scenes matching criteria")

### Inspect Scene Metadata

In [ ]:
# Extract metadata for all scenes
def extract_metadata(image):
    return ee.Feature(None, {
        'scene_id': image.get('PRODUCT_ID'),
        'date': image.date().format('YYYY-MM-dd'),
        'cloud_cover': image.get('CLOUDY_PIXEL_PERCENTAGE'),
        'solar_azimuth': image.get('MEAN_SOLAR_AZIMUTH_ANGLE'),
        'solar_zenith': image.get('MEAN_SOLAR_ZENITH_ANGLE')
    })

metadata = collection.map(extract_metadata).getInfo()['features']
df_scenes = pd.DataFrame([f['properties'] for f in metadata])

print("\n📊 Available Scenes:")
print(df_scenes.to_string(index=False))

In [ ]:
# Sort by cloud cover and select best scenes
df_scenes_sorted = df_scenes.sort_values('cloud_cover')
print("\n🌤️ Top 5 Clearest Scenes:")
print(df_scenes_sorted.head().to_string(index=False))

### Visualize Best Scene

In [ ]:
# Get the clearest image
best_image = ee.Image(collection.sort('CLOUDY_PIXEL_PERCENTAGE').first())

# Visualization parameters for True Color (RGB)
vis_params_rgb = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],  # Red, Green, Blue
    'gamma': 1.4
}

# Visualization for False Color (NIR, Red, Green) - highlights vegetation
vis_params_nir = {
    'min': 0,
    'max': 3000,
    'bands': ['B8', 'B4', 'B3'],  # NIR, Red, Green
    'gamma': 1.4
}

# Create map with both visualizations
Map = geemap.Map(center=[64.3, -21.15], zoom=11)
Map.addLayer(best_image, vis_params_rgb, 'True Color (RGB)')
Map.addLayer(best_image, vis_params_nir, 'False Color (NIR)', shown=False)
Map.addLayer(aoi, {'color': 'red'}, 'AOI', opacity=0.3)

# Add scene info
scene_date = best_image.date().format('YYYY-MM-dd').getInfo()
cloud_pct = best_image.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()
print(f"\n🖼️ Displaying Best Scene:")
print(f"   Date: {scene_date}")
print(f"   Cloud Cover: {cloud_pct:.1f}%")

Map

## Section 5: Select and Export Scenes (10 min)

### Select Multiple Scenes for Training
For ML, we want temporal diversity:

In [ ]:
# Select 4 scenes across the season
num_scenes = 4
selected_collection = collection.sort('CLOUDY_PIXEL_PERCENTAGE').limit(num_scenes)

# Get dates of selected scenes
selected_dates = selected_collection.aggregate_array('system:time_start').getInfo()
selected_dates = [datetime.datetime.fromtimestamp(d/1000).strftime('%Y-%m-%d') for d in selected_dates]

print(f"\n✅ Selected {num_scenes} scenes for dataset:")
for i, date in enumerate(selected_dates, 1):
    print(f"   {i}. {date}")

### Prepare Export Configuration
We'll export pre-processed imagery for the ML pipeline:

In [ ]:
# Select bands for ML (10m and 20m resampled to 10m)
ml_bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']  # RGB + NIR + SWIR

# Cloud masking function
def mask_clouds(image):
    """Mask clouds using SCL band (Scene Classification Layer)"""
    scl = image.select('SCL')
    # Keep clear (4,5,6) and exclude clouds (8,9,10)
    mask = scl.neq(8).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(3))
    return image.updateMask(mask).select(ml_bands)

# Apply cloud masking
masked_collection = selected_collection.map(mask_clouds)

print("✅ Applied cloud masking to selected scenes")

### Export to Google Drive (Alternative: Local Download)

For small AOIs, we can download directly. For larger areas, export to Google Drive:

In [ ]:
# Method 1: Export to Google Drive (for larger areas)
# Note: This triggers an export task, not immediate download

def export_to_drive(image, description):
    """Export single image to Google Drive"""
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder='iceland_ml_course',
        region=aoi,
        scale=10,  # 10m resolution
        crs='EPSG:4326',
        maxPixels=1e9
    )
    task.start()
    return task

# Export each scene
print("🚀 Starting export tasks...\n")
tasks = []
for i, date in enumerate(selected_dates, 1):
    img = ee.Image(masked_collection.toList(num_scenes).get(i-1))
    task = export_to_drive(img, f'sentinel2_iceland_{date.replace("-", "")}')
    tasks.append(task)
    print(f"   ✓ Task {i} started: {date}")

print(f"\n📌 Monitor progress at: https://code.earthengine.google.com/tasks")
print(f"📂 Files will appear in Google Drive: iceland_ml_course/")

In [ ]:
# Method 2: Direct download for small AOI (faster for this lab)
import requests
import os

# Save to project directory
output_dir = Path(os.getenv('PROJECT_training2600')) / 'my_workspace' / 'data' / 'sentinel2'
output_dir.mkdir(parents=True, exist_ok=True)

print(f"💾 Downloading scenes to: {output_dir}\n")

for i, date in enumerate(selected_dates[:2], 1):  # Download first 2 for speed
    img = ee.Image(masked_collection.toList(num_scenes).get(i-1))
    
    # Get download URL
    url = img.getDownloadURL({
        'region': aoi,
        'scale': 10,
        'format': 'GEO_TIFF'
    })
    
    # Download
    filename = output_dir / f'sentinel2_{date.replace("-", "")}.tif'
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)
    
    print(f"   ✓ Downloaded: {filename.name}")

print(f"\n✅ Download complete!")

### Save Scene Metadata

In [ ]:
# Save metadata for reference
metadata_file = output_dir / 'scene_metadata.csv'
df_scenes_sorted.head(num_scenes).to_csv(metadata_file, index=False)

print(f"📝 Saved metadata to: {metadata_file}")
print(f"\n{df_scenes_sorted.head(num_scenes).to_string(index=False)}")

## Summary & Next Steps

### What We Covered
✅ Set up Google Earth Engine authentication  
✅ Defined an AOI in Iceland  
✅ Queried Sentinel-2 imagery with filters  
✅ Visualized and selected optimal scenes  
✅ Exported/downloaded imagery for ML pipeline  

### Data Acquired
- **Scenes:** 4 Sentinel-2 images (Summer 2024)
- **Bands:** B2, B3, B4, B8, B11, B12 (6 bands)
- **Resolution:** 10m
- **Format:** GeoTIFF
- **Cloud Cover:** < 20%

### Key GEE Concepts
- **ImageCollection:** Time series of satellite images
- **Filtering:** Spatial, temporal, and attribute filters
- **Cloud Masking:** Remove cloudy pixels
- **Export:** Server-side processing for large areas

### Prepare for Lab 4
Next lab: **Data Preprocessing & Patch Extraction**
- We'll load the downloaded imagery
- Extract patches for training
- Apply normalization
- Match with CORINE land cover labels

### Additional Resources
- **GEE Catalog:** https://developers.google.com/earth-engine/datasets
- **Sentinel-2 Bands:** https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/resolutions/radiometric
- **geemap Tutorials:** https://geemap.org/tutorials

### Homework (Optional)
1. Try different AOIs in Iceland (coast, urban, volcanic)
2. Experiment with different date ranges
3. Compare cloud-free scenes across seasons
4. Calculate vegetation indices (NDVI, EVI)

---

**Excellent work!** You've successfully acquired real satellite data! 🛰️

---

## 🔵 OPTIONAL: Advanced Topics (For Early Finishers)

## 🔵 Advanced Topic 1: Spectral Indices

### Why Calculate Indices?
Spectral indices highlight specific features:
- Vegetation health (NDVI)
- Water bodies (NDWI)
- Burn severity (NBR)

### Common Indices

In [ ]:
# Calculate NDVI (Normalized Difference Vegetation Index)
# NDVI = (NIR - Red) / (NIR + Red)
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Calculate NDWI (Normalized Difference Water Index)
# NDWI = (Green - NIR) / (Green + NIR)
def add_ndwi(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return image.addBands(ndwi)

# Calculate NBR (Normalized Burn Ratio)
# NBR = (NIR - SWIR2) / (NIR + SWIR2)
def add_nbr(image):
    nbr = image.normalizedDifference(['B8', 'B12']).rename('NBR')
    return image.addBands(nbr)

# Apply to collection
collection_with_indices = collection.map(add_ndvi).map(add_ndwi).map(add_nbr)

# Get one image and visualize NDVI
image_ndvi = collection_with_indices.first()

# Visualization parameters for NDVI
ndvi_viz = {
    'bands': ['NDVI'],
    'min': -1,
    'max': 1,
    'palette': ['brown', 'yellow', 'green', 'darkgreen']
}

# Display (if using geemap)
# Map.addLayer(image_ndvi, ndvi_viz, 'NDVI')

print("Spectral indices calculated and added to images")

## 🔵 Advanced Topic 2: Cloud Masking with QA Bands

### QA60 Band
Sentinel-2 includes a QA60 band for cloud detection:
- Bit 10: Opaque clouds
- Bit 11: Cirrus clouds

In [ ]:
# Cloud masking function using QA60
def mask_s2_clouds(image):
    qa = image.select('QA60')
    
    # Bits 10 and 11 are clouds and cirrus
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    
    # Both flags should be zero (no clouds)
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
           qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    
    return image.updateMask(mask)

# Apply cloud mask to collection
collection_masked = collection.map(mask_s2_clouds)

# Compare before and after
image_original = collection.first()
image_masked = collection_masked.first()

print("Cloud masking applied")
print("Original image bands:", image_original.bandNames().getInfo())
print("Masked image bands:", image_masked.bandNames().getInfo())

## 🔵 Advanced Topic 3: Temporal Analysis

### Time Series Analysis
Analyze how the landscape changes over time

In [ ]:
# Query images from different seasons
winter_collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(aoi) \
    .filterDate('2023-12-01', '2024-02-28') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

summer_collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(aoi) \
    .filterDate('2023-06-01', '2023-08-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Create median composites
winter_median = winter_collection.median()
summer_median = summer_collection.median()

print(f"Winter images: {winter_collection.size().getInfo()}")
print(f"Summer images: {summer_collection.size().getInfo()}")

# Calculate NDVI difference
winter_ndvi = winter_median.normalizedDifference(['B8', 'B4'])
summer_ndvi = summer_median.normalizedDifference(['B8', 'B4'])
ndvi_diff = summer_ndvi.subtract(winter_ndvi)

print("Seasonal comparison completed")

## 🔵 Advanced Topic 4: Multiple AOIs

### Compare Different Regions

In [ ]:
# Define multiple AOIs in Iceland
aois = {
    'thingvellir': ee.Geometry.Rectangle([-21.2, 64.2, -21.0, 64.3]),
    'reykjavik': ee.Geometry.Rectangle([-22.0, 64.1, -21.8, 64.2]),
    'vatnajokull': ee.Geometry.Rectangle([-16.8, 64.3, -16.3, 64.5])
}

# Query each AOI
for name, aoi_geom in aois.items():
    aoi_collection = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(aoi_geom) \
        .filterDate('2023-06-01', '2023-09-30') \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    
    count = aoi_collection.size().getInfo()
    print(f"{name}: {count} images available")
    
    if count > 0:
        # Get median composite
        median_image = aoi_collection.median()
        print(f"  Median composite created for {name}")

## 🔵 Advanced Topic 5: Batch Export

### Export Multiple Scenes Automatically

In [ ]:
# Export all images in collection (up to 10)
image_list = collection.toList(10)  # Limit to 10 images
count = min(collection.size().getInfo(), 10)

for i in range(count):
    image = ee.Image(image_list.get(i))
    
    # Get image ID and date
    image_id = image.get('system:index').getInfo()
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    
    # Export task
    task = ee.batch.Export.image.toDrive(
        image=image.select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']),
        description=f'S2_{date}_{i}',
        folder='GEE_Exports',
        fileNamePrefix=f'iceland_s2_{date}',
        region=aoi,
        scale=10,
        maxPixels=1e13,
        fileFormat='GeoTIFF'
    )
    
    task.start()
    print(f"Export task {i+1} started: {date}")

print(f"\n{count} export tasks submitted to Google Drive")

---

## ✅ Lab 3 Completion Checklist

### Core Tasks (Must Complete)
- [ ] GEE account authenticated
- [ ] AOI defined in Iceland
- [ ] Sentinel-2 collection queried
- [ ] Cloud filtering applied (<20%)
- [ ] Visualized RGB and false color
- [ ] Exported 4 scenes to Google Drive
- [ ] Downloaded scenes to local/JURECA

### Optional Tasks (If Time Permits)
- [ ] Calculated spectral indices (NDVI, NDWI, NBR)
- [ ] Applied advanced cloud masking
- [ ] Created seasonal composites
- [ ] Compared multiple AOIs
- [ ] Set up batch export

---

## 📝 Homework / Async Learning
- Download all exported scenes from Google Drive
- Transfer scenes to JURECA: `$PROJECT_training2600/data/sentinel2/`
- Explore GEE Code Editor: https://code.earthengine.google.com
- Try different AOIs and date ranges

## 🚀 Next Lab Preview
**Lab 4: Data Preprocessing**
- Load Sentinel-2 GeoTIFFs with rasterio
- Apply normalization techniques
- Align imagery with land cover labels
- Create train/val/test splits

**Bring to Next Lab:**
- Downloaded Sentinel-2 scenes
- AOI definition file